In [3]:
import networkit as nk
import graph_tool.all as gt

In [4]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

In [5]:
import json
import powerlaw

In [6]:
import os
import time

## Setup

Since the network fits into memory, *laughs in 128GB of RAM* :), we will work with both the `netowrkit` and the `graph-tool` representation. To **ensure** consistency, `networkit` reads the file that comes from `transform.ipynb`, and checks there are no unexpected errors such as, in this case, more than 1 connected component and if the graph is consistent. 

After the verifications it saves the possibly modified graph into both GraphML and CSV formats into the `temp` folder. The network in this folder will be read by `graph-tool`.

In [7]:
sns.set_style("whitegrid")
sns.set_context("paper")

In [8]:
basic_statistics = {}
degree_distr_stats = {}

In [9]:
CELLS_RUNNED = np.zeros(16)

In [10]:
CELLS_RUNNED[0] = 1
####################

NOM_COLOR = 'red'
ER_COLOR = 'blue'
BA_COLOR = 'green'

COLOR_ONE = '#346751'
COLOR_TWO = '#C84B31'
COLOR_THREE = '#E5A5FF'

In [11]:
CELLS_RUNNED[1] = 1
####################

BASE = os.path.abspath(os.curdir)

ORIGINAL_NETWORK_NAME_GML = "output_graph_no_cliques2.graphml"
ORIGINAL_NETWORK_NAME_CSV = "output_graph_no_cliques2.csv"
TEMP_NETWORKS_FOLDER = "temp"
PLOTS_FOLDER = "plots"
STATS_FOLDER = "stats"
RAW_GRAPH_FOLDER = "all_ranks_graph"

PATH_TO_NETWORK_GML = os.path.join(BASE, RAW_GRAPH_FOLDER, "processed", ORIGINAL_NETWORK_NAME_GML)
PATH_TO_NETWORK_CSV = os.path.join(BASE, RAW_GRAPH_FOLDER, "processed", ORIGINAL_NETWORK_NAME_CSV)

PATH_TO_STATS_FOLDER = os.path.join(BASE, STATS_FOLDER)
PATH_TO_TEMP_FOLDER = os.path.join(BASE, TEMP_NETWORKS_FOLDER)
PATH_TO_PLOTS_FOLDER = os.path.join(BASE, PLOTS_FOLDER)

if not os.path.exists(PATH_TO_STATS_FOLDER):
    os.makedirs(PATH_TO_STATS_FOLDER)

if not os.path.exists(PATH_TO_TEMP_FOLDER):
    os.makedirs(PATH_TO_TEMP_FOLDER)
    
if not os.path.exists(PATH_TO_PLOTS_FOLDER):
    os.makedirs(PATH_TO_PLOTS_FOLDER)

In [12]:
CELLS_RUNNED[2] = 1
####################

G = nk.graphio.EdgeListReader(separator=",", firstNode=0, continuous=True).read(PATH_TO_NETWORK_CSV)

In [13]:
CELLS_RUNNED[3] = 1
####################

def preamble_of_G():
    n_nodes = G.numberOfNodes()
    n_edges = G.numberOfEdges()
    print("Is it direct? ", G.isDirected())
    print("Number of nodes: ", n_nodes)
    print("Number of edges: ", n_edges)
    return n_nodes, n_edges

n_nodes, n_edges = preamble_of_G()

Is it direct?  False
Number of nodes:  44587
Number of edges:  779615


In [14]:
CELLS_RUNNED[4] = 1
####################

components = nk.components.ConnectedComponents(G).run()
connected_components = components.numberOfComponents()
if connected_components != 1:
    print("More than 1 component, extracting largest one.")
    G = components.extractLargestConnectedComponent(G)
    n_nodes, n_edges = preamble_of_G()
    print("New Number of CC: ", nk.components.ConnectedComponents(G).run().numberOfComponents())
    basic_statistics["og_cc_number"] = connected_components
    
if not G.checkConsistency():
    print("Making it consistent.")
    G.removeMultiEdges()
    G.compactEdges()
    G.removeSelfLoops()
    G.compactEdges()
    n_nodes, n_edges = preamble_of_G()
    
writergml = nk.graphio.GraphMLWriter()
writergml.write(G, os.path.join(PATH_TO_TEMP_FOLDER, "temp.graphml"))
writecsv = nk.graphio.EdgeListWriter(",", 0, bothDirections=False)
writecsv.write(G, os.path.join(PATH_TO_TEMP_FOLDER, "temp.csv"))
    
basic_statistics["n_nodes"] = n_nodes
basic_statistics["n_edges"] = n_edges

In [15]:
CELLS_RUNNED[5] = 1
####################

# g = gt.load_graph_from_csv(PATH_TO_NETWORK, csv_options={'delimiter': ',', 'quotechar': '"'})
g = gt.load_graph(os.path.join(PATH_TO_TEMP_FOLDER, "temp.graphml"))
g = gt.extract_largest_component(g, directed=False, prune=True)

In [16]:
CELLS_RUNNED[6] = 1
####################

assert g.num_vertices() == G.numberOfNodes() and g.num_edges() == G.numberOfEdges()

---

## Statistics

In this section we calculate statistics related with the degree of the network. We start by looking at plots for the degree distribution and fitting a powerlaw to it. 

We also calculate other statistics related to the overall topology of the network. 

In [15]:
CELLS_RUNNED[7] = 1
####################

dd = sorted(nk.centrality.DegreeCentrality(G).run().scores(), reverse=True)
degrees, count = np.unique(dd, return_counts=True)

rel_freq = count/sum(count)
cumulative = []
for i in range(len(degrees)):
    cumulative.append(sum(rel_freq[i:]))

plt.figure(figsize=(11,7))
plt.xscale('log')
plt.xlabel("Degree")
plt.yscale('log')
plt.ylabel("Number of Nodes")
fig = sns.scatterplot(x=degrees, y=count, 
                hue=np.repeat("Linear Binning", len(degrees)), palette=[COLOR_ONE], 
                edgecolor="black", linewidth=0.3).get_figure()
fig.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_lin_bin.pdf"))
plt.close()

plt.figure(figsize=(11,7))
plt.xscale('log')
plt.xlabel("Degree")
plt.yscale('log')
plt.ylabel("Frequency")
fig = sns.scatterplot(x=degrees, y=cumulative,
                hue=np.repeat("Cumulative Binning", len(degrees)), palette=[COLOR_TWO], 
                edgecolor="black", linewidth=0.3).get_figure()
fig.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_cum_bin.pdf"))
plt.close()

degree_distribution = pd.DataFrame({"Degree": degrees, "Count": count})
degree_distribution.to_csv(os.path.join(PATH_TO_STATS_FOLDER, RAW_GRAPH_FOLDER + "_degree_distribution.csv"), index=False)

In [17]:
CELLS_RUNNED[8] = 1
####################

fitted = powerlaw.Fit(dd, discrete=True, fit_method="Likelihood")
print("Alpha: ", fitted.alpha, " XMin: ", fitted.xmin)
print("Alpha (TPL): ", fitted.truncated_power_law.alpha, ", Lambda (TPL): ", fitted.truncated_power_law.Lambda, ", XMin: ", fitted.xmin)
print("Mu: ", fitted.lognormal.mu, ", Sigma: ", fitted.lognormal.sigma, ", XMin: ", fitted.xmin)


ll_test_pw_exp = fitted.loglikelihood_ratio('power_law', "exponential", normalized_ratio=True)
print("[PL vs EXP] LogLikelihood Ratio: {0}, p-value: {1}".format(ll_test_pw_exp[0], ll_test_pw_exp[1]))

ll_test_pw_ln = fitted.loglikelihood_ratio('power_law', "lognormal", normalized_ratio=True)
print("[PL vs LN] LogLikelihood Ratio: {0}, p-value: {1}".format(ll_test_pw_ln[0], ll_test_pw_ln[1]))

ll_test_ln_tpl = fitted.loglikelihood_ratio('lognormal', "truncated_power_law", normalized_ratio=True)
print("[LN vs TPL] LogLikelihood Ratio: {0}, p-value: {1}".format(ll_test_ln_tpl[0], ll_test_ln_tpl[1]))

ll_test_tpl_se = fitted.loglikelihood_ratio('truncated_power_law', "stretched_exponential", normalized_ratio=True)
print("[TPL vs SE] LogLikelihood Ratio: {0}, p-value: {1}".format(ll_test_tpl_se[0], ll_test_tpl_se[1]))

degree_distr_stats["xmin"] = fitted.xmin

degree_distr_stats["alpha_pl"] = fitted.alpha
degree_distr_stats["alpha_tpl"] = fitted.truncated_power_law.alpha
degree_distr_stats["lambda_tpl"] = fitted.truncated_power_law.Lambda
degree_distr_stats["mu_tpl"] = fitted.lognormal.mu
degree_distr_stats["sigma_tpl"] = fitted.lognormal.sigma
degree_distr_stats["lambda_se"] = fitted.stretched_exponential.Lambda

degree_distr_stats["llratio_pl_exp"] = ll_test_pw_exp[0]
degree_distr_stats["p_value_pl_exp"] = ll_test_pw_exp[1]

degree_distr_stats["llratio_pl_ln"] = ll_test_pw_ln[0]
degree_distr_stats["p_value_pl_ln"] = ll_test_pw_ln[1]

degree_distr_stats["llratio_ln_tpl"] = ll_test_ln_tpl[0]
degree_distr_stats["p_value_ln_tpl"] = ll_test_ln_tpl[1]

degree_distr_stats["llratio_tpl_se"] = ll_test_tpl_se[0]
degree_distr_stats["p_value_tpl_se"] = ll_test_tpl_se[1]

Calculating best minimal value for power law fit
Alpha:  3.1168812638097956  XMin:  48.0
Alpha (TPL):  2.4730378789905134 , Lambda (TPL):  0.005005829910114423 , XMin:  48.0
Mu:  1.710026080569207 , Sigma:  1.1910160873713245 , XMin:  48.0
[PL vs EXP] LogLikelihood Ratio: 7.61869372667599, p-value: 2.5625567692003894e-14
[PL vs LN] LogLikelihood Ratio: -6.525676773547729, p-value: 6.769516479060339e-11
[LN vs TPL] LogLikelihood Ratio: -2.893969909453835, p-value: 0.0038040461470476416
[TPL vs SE] LogLikelihood Ratio: 2.353837265817692, p-value: 0.01858074351963828


In [ ]:
CELLS_RUNNED[15] = 1
####################
plt.figure(figsize=(11,7))

plt.xlabel("Degree")
plt.ylabel("Frequency")
powerlaw.plot_pdf(dd, color="black", alpha=0.8)
fitted.power_law.plot_pdf(color=COLOR_TWO, linestyle="dashdot", alpha=1, label="Power Law")
fitted.lognormal.plot_pdf(color=COLOR_THREE, linestyle="dashed", alpha=1, label="LogNormal")
fitted.truncated_power_law.plot_pdf(color=COLOR_ONE, linestyle="dotted", alpha=1, label="Truncated Power Law")
plt.legend()

plt.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_fitted_distributions.pdf"))
plt.close()

In [68]:
CELLS_RUNNED[9] = 1
####################

local_clustering_coefficient = nk.centrality.LocalClusteringCoefficient(G).run().scores()

basic_statistics["avg_clust_coefficient"] = (np.mean(local_clustering_coefficient), np.std(local_clustering_coefficient))
print("AVG Clustering Coefficient: ", basic_statistics["avg_clust_coefficient"])

AVG Clustering Coefficient:  (0.33818439117279253, 0.18550680874146552)


In [69]:
CELLS_RUNNED[10] = 1
####################

gt.vertex_average(g, "total")

(34.97050709848162, 0.16815089175692952)

In [70]:
CELLS_RUNNED[11] = 1
####################

degree_distr_stats["average_degree"] = gt.vertex_average(g, "total") # Do not count 0 degree
basic_statistics["pseudo_diameter"] = gt.pseudo_diameter(g)[0]

dist = gt.shortest_distance(g)
ave_path_length = sum([sum(i) for i in dist])/(g.num_vertices()**2-g.num_vertices())
basic_statistics["avg_path_length"] = ave_path_length

In [71]:
CELLS_RUNNED[12] = 1
####################

tra, n_triangles, n_triples = gt.global_clustering(g, ret_counts=True) # Transitivity

basic_statistics["transitivity"] = tra
basic_statistics["n_triangles"] = n_triangles
basic_statistics["n_triples"] = n_triples

---

## Centralities

Extract normalized betweenness, closeness and eigenvector centrality.

In [72]:
CELLS_RUNNED[13] = 1
####################

# Exact centralities (HEAVY WORK)
start = time.time()
bet_vpm, bet_epm = gt.betweenness(g, norm=True)
elapsed = time.time() - start
print("GT for Betweenness time : " + str(elapsed))

start = time.time()
clos_v = gt.closeness(g, norm=True) 
elapsed = time.time() - start
print("GT for Closeness time : " + str(elapsed))

start = time.time()
eigenvalue, eigenvector = gt.eigenvector(g)
elapsed = time.time() - start
print("GT for Eigenvector time : " + str(elapsed))

GT for Betweenness time : 107.04376363754272
GT for Closeness time : 43.869438886642456
GT for Eigenvector time : 0.02357935905456543


In [73]:
CELLS_RUNNED[14] = 1
####################

betw_value = []
clos_value = []
eign_value = []
i, j, k = 0, 0, 0
for v in bet_vpm:
    betw_value.append(v)
    i += 1

for v in clos_v:
    clos_value.append(v)
    j += 1

l2_norm = 0
for v in eigenvector:  
    eign_value.append(v)
    l2_norm += v**2
    k += 1
l2_norm = np.sqrt(l2_norm)
    
assert (i==j and j==k)
    
vertex_id = np.arange(i)
centralities = pd.DataFrame({"vertex_id": vertex_id, "betweenness": betw_value, "closeness": clos_value,
                             "eigenvector": eign_value/l2_norm})
centralities.to_csv(os.path.join(PATH_TO_STATS_FOLDER, RAW_GRAPH_FOLDER+"_centralities.csv"), index=False)

---

In [74]:
assert all(CELLS_RUNNED)

with open(os.path.join(PATH_TO_STATS_FOLDER, RAW_GRAPH_FOLDER+"_degree_stats.json"), 'w') as fp:
    json.dump(degree_distr_stats, fp)
    
with open(os.path.join(PATH_TO_STATS_FOLDER, RAW_GRAPH_FOLDER+"_basic_stats.json"), 'w') as fp:
    json.dump(basic_statistics, fp)
    
CELLS_RUNNED = np.zeros(15)

---

## Tests Against Theoretical Networks

In this section we will produce code to compare the statistics obtained with a null model (ER network) and a BA model.

In [18]:
with open(os.path.join(PATH_TO_STATS_FOLDER, RAW_GRAPH_FOLDER+"_degree_stats.json")) as fp:
    degree_distr_stats = json.load(fp)

### Degree Distribution Comparison

In [19]:
from scipy.stats import poisson

count_poisson = []
for d in degrees:
    count_poisson.append(poisson.pmf(d, mu=degree_distr_stats['average_degree'][0])*g.num_vertices())
    
# sorted([(d,c) for d,c in zip(degrees, count_poisson)], key=lambda d: d[0])

In [20]:
degree_clean = []
count_clean = []

for d,c in zip(degrees, count_poisson):
    if c >= 1:
        degree_clean.append(d)
        count_clean.append(c)

In [21]:
p = (degree_distr_stats["average_degree"][0] + degree_distr_stats["average_degree"][1])/(g.num_vertices()-1)
er = nk.generators.ErdosRenyiGenerator(g.num_vertices(), prob=p)
erg = er.generate()

erg_dd = sorted(nk.centrality.DegreeCentrality(erg).run().scores(), reverse=True)
erg_degrees, erg_count = np.unique(erg_dd, return_counts=True)

In [80]:
ba = nk.generators.BarabasiAlbertGenerator(18, g.num_vertices(), n0=0, batagelj=False)
bag = ba.generate()

bag_dd = sorted(nk.centrality.DegreeCentrality(bag).run().scores(), reverse=True)
bag_degrees, bag_count = np.unique(bag_dd, return_counts=True)


In [138]:
plt.figure(figsize=(11,7))
sns.scatterplot(x=degree_clean, y=count_clean, 
                hue=np.repeat("Theoretical Poisson", len(degree_clean)), palette=[COLOR_THREE], 
                edgecolor="black", linewidth=0.2, alpha=1)

sns.scatterplot(x=erg_degrees, y=erg_count, 
                hue=np.repeat("Erdos-Renyi", len(erg_degrees)), palette=[ER_COLOR], 
                edgecolor="black", linewidth=0.3, alpha=1)

plt.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_poisson_er.pdf"))
plt.close()

In [141]:
plt.figure(figsize=(11,7))
plt.xscale('log')
plt.xlabel("Degree")
plt.yscale('log')
plt.ylabel("Number of Nodes")


sns.scatterplot(x=bag_degrees, y=bag_count, 
                hue=np.repeat("BA", len(bag_degrees)), palette=[BA_COLOR], 
                edgecolor="black", linewidth=0.3, alpha=1, marker="o")

sns.scatterplot(x=degrees, y=count, 
                hue=np.repeat("NoM", len(degrees)), palette=[NOM_COLOR], 
                edgecolor="black", linewidth=0.3, alpha=1, marker='.')

plt.axvline(x=48, color="black") # xmin
plt.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_degree_dis_theory_real.pdf"))
plt.close()

In [125]:
clustering_degree = list(zip(local_clustering_coefficient, nk.centrality.DegreeCentrality(G).run().scores()))
clustering_degree_sorted = sorted([(c, d) for c,d in clustering_degree], key=lambda kp: kp[1])
cl = [c for c,_ in clustering_degree_sorted]
de = [d for _,d in clustering_degree_sorted]

clustering_degree_erg = list(zip(nk.centrality.LocalClusteringCoefficient(erg).run().scores(),
                             nk.centrality.DegreeCentrality(erg).run().scores()))
clustering_degree_sorted_erg = sorted([(c, d) for c,d in clustering_degree_erg], key=lambda kp: kp[1])
cl_erg = [c for c,_ in clustering_degree_sorted_erg]
de_erg = [d for _,d in clustering_degree_sorted_erg]

plt.figure(figsize=(11,7))
plt.xscale('log')
plt.xlabel("Degree")
plt.yscale('log')
plt.ylabel("Clustering Coefficient")
fig = sns.scatterplot(x=de_erg, y=cl_erg,
                hue=np.repeat("Clust", len(de)), palette=[ER_COLOR], 
                edgecolor="black", linewidth=0.1, alpha=0.7, legend=None).get_figure()
fig.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_clustering_er.pdf"))
plt.close()

plt.figure(figsize=(11,7))
plt.xscale('log')
plt.xlabel("Degree")
plt.yscale('log')
plt.ylabel("Clustering Coefficient")
fig = sns.scatterplot(x=de, y=cl,
                hue=np.repeat("Clust", len(de)), palette=[NOM_COLOR], 
                edgecolor="black", linewidth=0.1, alpha=0.7, legend=None).get_figure()
fig.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_clustering_nom.pdf"))
plt.close()

In [114]:
ba_high = nk.generators.BarabasiAlbertGenerator(58, g.num_vertices(), n0=0, batagelj=False)
bag_high = ba_high.generate()
bag_dd_high = sorted(nk.centrality.DegreeCentrality(bag_high).run().scores(), reverse=True)
bag_degrees_high, bag_count_high = np.unique(bag_dd_high, return_counts=True)

ba_low = nk.generators.BarabasiAlbertGenerator(3, g.num_vertices(), n0=0, batagelj=False)
bag_low = ba_low.generate()
bag_dd_low = sorted(nk.centrality.DegreeCentrality(bag_low).run().scores(), reverse=True)
bag_degrees_low, bag_count_low = np.unique(bag_dd_low, return_counts=True)

In [115]:
fitted = powerlaw.Fit(bag_dd_high, discrete=True, fit_method="Likelihood")
print("Alpha: ", fitted.alpha, " XMin: ", fitted.xmin)
print("Alpha (TPL): ", fitted.truncated_power_law.alpha, ", Lambda (TPL): ", fitted.truncated_power_law.Lambda, ", XMin: ", fitted.xmin)
print("Mu: ", fitted.lognormal.mu, ", Sigma: ", fitted.lognormal.sigma, ", XMin: ", fitted.xmin)

fitted = powerlaw.Fit(bag_dd_low, discrete=True, fit_method="Likelihood")
print("Alpha: ", fitted.alpha, " XMin: ", fitted.xmin)
print("Alpha (TPL): ", fitted.truncated_power_law.alpha, ", Lambda (TPL): ", fitted.truncated_power_law.Lambda, ", XMin: ", fitted.xmin)
print("Mu: ", fitted.lognormal.mu, ", Sigma: ", fitted.lognormal.sigma, ", XMin: ", fitted.xmin)

Calculating best minimal value for power law fit
Alpha:  2.96367228046989  XMin:  58.0
Alpha (TPL):  2.9468803549591063 , Lambda (TPL):  7.571740913693468e-05 , XMin:  58.0
Mu:  -246.90278643389308 , Sigma:  11.329551564219031 , XMin:  58.0
Calculating best minimal value for power law fit
Alpha:  2.773140100776507  XMin:  6.0
Alpha (TPL):  2.7390938277592136 , Lambda (TPL):  0.0022582407457654224 , XMin:  6.0
Mu:  -11.271367001340217 , Sigma:  2.805622527480198 , XMin:  6.0


In [116]:
writergml.write(bag_high, os.path.join(PATH_TO_TEMP_FOLDER, "temp_ba_high.graphml"))
writergml.write(bag_low, os.path.join(PATH_TO_TEMP_FOLDER, "temp_ba_low.graphml"))

In [17]:
bag_low = gt.load_graph(os.path.join(PATH_TO_TEMP_FOLDER, "temp_ba_low.graphml"))
bag_high = gt.load_graph(os.path.join(PATH_TO_TEMP_FOLDER, "temp_ba_high.graphml"))

---

### Centrality Comparisons

In [24]:
# Exact centralities (HEAVY WORK)
start = time.time()
bet_vpm, bet_epm = gt.betweenness(ger, norm=True)
elapsed = time.time() - start
print("GT for Betweenness time : " + str(elapsed))

start = time.time()
clos_v = gt.closeness(ger, norm=True) 
elapsed = time.time() - start
print("GT for Closeness time : " + str(elapsed))

start = time.time()
eigenvalue, eigenvector = gt.eigenvector(ger)
elapsed = time.time() - start
print("GT for Eigenvector time : " + str(elapsed))

betw_value = []
clos_value = []
eign_value = []
i, j, k = 0, 0, 0
for v in bet_vpm:
    betw_value.append(v)
    i += 1

for v in clos_v:
    clos_value.append(v)
    j += 1

l2_norm = 0
for v in eigenvector:  
    eign_value.append(v)
    l2_norm += v**2
    k += 1
l2_norm = np.sqrt(l2_norm)
    
assert (i==j and j==k)

GT for Betweenness time : 138.67091250419617
GT for Closeness time : 48.03222703933716
GT for Eigenvector time : 0.008581876754760742


In [25]:
vertex_id = np.arange(i)
centralities_nom = pd.read_csv(os.path.join(PATH_TO_STATS_FOLDER, RAW_GRAPH_FOLDER+"_centralities.csv"))
centralities_ger = pd.DataFrame({"vertex_id": vertex_id, "betweenness": betw_value, "closeness": clos_value,
                             "eigenvector": eign_value/l2_norm})

In [26]:
joined = centralities_nom.join(centralities_ger, lsuffix='_nom', rsuffix='_ger')
joined = joined.drop(["vertex_id_ger"], axis=1)

In [88]:
betweenness = pd.concat([joined["betweenness_nom"], joined["betweenness_ger"]], ignore_index=True)
closeness = pd.concat([joined["closeness_nom"], joined["closeness_ger"]], ignore_index=True)
eigenvector = pd.concat([joined["eigenvector_nom"], joined["eigenvector_ger"]], ignore_index=True)
centrality_type = np.repeat(["Betweenness", "Closeness", "Eigenvector"], betweenness.shape[0]) 
network_type = np.tile(np.repeat(["NoM", "ER"], betweenness.shape[0]//2), 3)

melted = pd.DataFrame({"Values": pd.concat([betweenness, closeness, eigenvector]),
                    "Centrality Type": centrality_type, "Network Type": network_type})

In [89]:
plt.figure(figsize=(11,7))
g = sns.FacetGrid(melted, row="Centrality Type", col="Network Type", 
                  hue="Network Type", palette=[NOM_COLOR, ER_COLOR], sharex=False, sharey=False)
g.map(sns.histplot, "Values", element="step")
g.set_titles(col_template="{col_name}", fontweight='bold', size=10)

plt.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_centralities.pdf"), bbox_inches="tight")
plt.close()

<Figure size 1100x700 with 0 Axes>

---

## Percolation 

In this section experiments related with percolation theory are done.

In [19]:
# vertices_plot = list(map(lambda v: v.out_degree(), sorted([v for v in g.vertices()], key=lambda v: -v.out_degree())))
vertices = sorted([v for v in g.vertices()], key=lambda v: v.out_degree())
vertices_bag_high = sorted([v for v in bag_high.vertices()], key=lambda v: v.out_degree())
vertices_bag_low = sorted([v for v in bag_low.vertices()], key=lambda v: v.out_degree())

In [20]:
sizes, comp = gt.vertex_percolation(g, vertices)
np.random.shuffle(vertices)
sizes2, comp = gt.vertex_percolation(g, vertices)

sizes_bag_high, comp_bag_high = gt.vertex_percolation(bag_high, vertices_bag_high)
np.random.shuffle(vertices_bag_high)
sizes2_bag_high, comp_bag_high = gt.vertex_percolation(bag_high, vertices_bag_high)

sizes_bag_low, comp_bag_low = gt.vertex_percolation(bag_low, vertices_bag_low)
np.random.shuffle(vertices_bag_low)
sizes2_bag_low, comp_bag_low = gt.vertex_percolation(bag_low, vertices_bag_low)

In [21]:
ds = pd.DataFrame({"Size Largest Component": np.append(sizes,sizes2), 
                   "Removal Order": np.repeat(["By Decreasing Degree", "Random"], len(sizes)),
                   "Vertices Remaining": np.append(np.arange(g.num_vertices()), np.arange(g.num_vertices()))})

ds_bag_high = pd.DataFrame({"Size Largest Component": np.append(sizes_bag_high,sizes2_bag_high), 
                   "Removal Order": np.repeat(["By Decreasing Degree", "Random"], len(sizes2_bag_high)),
                   "Vertices Remaining": np.append(np.arange(bag_high.num_vertices()), np.arange(bag_high.num_vertices()))})

ds_bag_low = pd.DataFrame({"Size Largest Component": np.append(sizes_bag_low,sizes2_bag_low), 
                   "Removal Order": np.repeat(["By Decreasing Degree", "Random"], len(sizes2_bag_low)),
                   "Vertices Remaining": np.append(np.arange(bag_low.num_vertices()), np.arange(bag_low.num_vertices()))})

In [22]:
fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(11,8))


sns.lineplot(data=ds_bag_high, x='Vertices Remaining', y='Size Largest Component', hue="Removal Order", 
             style="Removal Order", palette=[COLOR_ONE, ER_COLOR], markers=False, ax=ax1)
ax1.title.set_text("BA - Alpha ~ 3, kmin = 58")

sns.lineplot(data=ds, x='Vertices Remaining', y='Size Largest Component', hue="Removal Order", 
             style="Removal Order", palette=[COLOR_TWO, ER_COLOR], markers=False, ax=ax2)
ax2.title.set_text("NoM - Alpha = 3.11, kmin = 48")

sns.lineplot(data=ds_bag_low, x='Vertices Remaining', y='Size Largest Component', hue="Removal Order", 
             style="Removal Order", palette=[COLOR_THREE, ER_COLOR], markers=False, ax=ax3)
ax3.title.set_text("BA - Alpha ~ 3, kmin = 6")

plt.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_site_percolation.pdf"), bbox_inches="tight")
plt.close()

In [23]:
edges = sorted([(e.source(), e.target()) for e in g.edges()],
                 key=lambda e: e[0].out_degree() * e[1].out_degree())

edges_high = sorted([(e.source(), e.target()) for e in bag_high.edges()],
                 key=lambda e: e[0].out_degree() * e[1].out_degree())

edges_low = sorted([(e.source(), e.target()) for e in bag_low.edges()],
                 key=lambda e: e[0].out_degree() * e[1].out_degree())

In [24]:
sizes, comp = gt.edge_percolation(g, edges)
np.random.shuffle(edges)
sizes2, comp = gt.edge_percolation(g, edges)

sizes_high, comp_high = gt.edge_percolation(bag_high, edges_high)
np.random.shuffle(edges_high)
sizes2_high, comp_high = gt.edge_percolation(bag_high, edges_high)

sizes_low, comp_low = gt.edge_percolation(bag_low, edges_low)
np.random.shuffle(edges_low)
sizes2_low, comp_low = gt.edge_percolation(bag_low, edges_low)

In [25]:
ds = pd.DataFrame({"Size Largest Component": np.append(sizes,sizes2), 
                   "Removal Order": np.repeat(["By Decreasing Degree", "Random"], len(sizes)),
                   "Edges Remaining": np.append(np.arange(g.num_edges()), np.arange(g.num_edges()))})

ds_bag_high = pd.DataFrame({"Size Largest Component": np.append(sizes_high,sizes2_high), 
                   "Removal Order": np.repeat(["By Decreasing Degree", "Random"], len(sizes2_high)),
                   "Edges Remaining": np.append(np.arange(bag_high.num_edges()), np.arange(bag_high.num_edges()))})

ds_bag_low = pd.DataFrame({"Size Largest Component": np.append(sizes_low,sizes2_low), 
                   "Removal Order": np.repeat(["By Decreasing Degree", "Random"], len(sizes_low)),
                   "Edges Remaining": np.append(np.arange(bag_low.num_edges()), np.arange(bag_low.num_edges()))})

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=False, figsize=(11,8))


sns.lineplot(data=ds_bag_high, x='Edges Remaining', y='Size Largest Component', hue="Removal Order", 
             style="Removal Order", palette=[COLOR_ONE, ER_COLOR], markers=False, ax=ax1)
ax1.title.set_text("BA - Alpha ~ 3, kmin = 58")

sns.lineplot(data=ds, x='Edges Remaining', y='Size Largest Component', hue="Removal Order", 
             style="Removal Order", palette=[COLOR_TWO, ER_COLOR], markers=False, ax=ax2)
ax2.title.set_text("NoM - Alpha = 3.11, kmin = 48")

sns.lineplot(data=ds_bag_low, x='Edges Remaining', y='Size Largest Component', hue="Removal Order", 
             style="Removal Order", palette=[COLOR_THREE, ER_COLOR], markers=False, ax=ax3)
ax3.title.set_text("BA - Alpha ~ 3, kmin = 6")
fig.tight_layout()

plt.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_bond_percolation.pdf"), bbox_inches="tight")
plt.close()

---

## Cascade Behaviour - SI/SIR Models

In [49]:
state = gt.SIRState(g, beta=0.056, gamma=0.38) # mu=0.38

In [50]:
S, X, R = [], [], []

for t in range(80):
    ret = state.iterate_sync()
    s = state.get_state().fa
    S.append((s == 0).sum())
    X.append((s == 1).sum())
    R.append((s == 2).sum())

In [51]:
plt.figure(figsize=(11, 7))
plt.plot(S, label="Normally Susceptible To Scam", c=COLOR_ONE)
plt.plot(X, label="Scammed", c=COLOR_TWO)
plt.plot(R, label="Immune to Scam", c=COLOR_THREE) # Hyper-aware to Scam
plt.legend(loc="center right")
plt.xlabel("Time")
plt.ylabel("Number of Nodes")

plt.savefig(os.path.join(PATH_TO_PLOTS_FOLDER, RAW_GRAPH_FOLDER + "_sir.pdf"), bbox_inches="tight")
plt.close()

---

## Motifs

In [ ]:
motifs, zscore = gt.motif_significance(g, 4, n_shuffles=50)

---